In [1]:
import torch 
import numpy as np
import transformers, datasets, accelerate, tensorboard, evaluate
from models import Student
import torch.nn as nn
import torch.optim as optim
from util import *
from loss import LossCalulcator
from pretrained_kd import *
from datasets import Array3D, ClassLabel, Features, load_dataset, Image
from matplotlib import pyplot
from tqdm import tqdm
from transformers import AdamW, ViTFeatureExtractor, ViTModel
from torch.profiler import profile, record_function, ProfilerActivity


In [2]:
# define hyperparameters
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
epochs = 50
lr = 0.001
lr_stepsize = 20
batch_size = 500
test_batch_size = 100
temperature=2
alpha=0.7


In [3]:
# Load model directly
from transformers import ViTForImageClassification, ViTFeatureExtractor
# from transformers import EfficientNetImageProcessor, EfficientNetForImageClassification


# import timm

feature_extractor = ViTFeatureExtractor.from_pretrained('nateraw/vit-base-patch16-224-cifar10')
teacher = ViTForImageClassification.from_pretrained('nateraw/vit-base-patch16-224-cifar10')


# processor = EfficientNetImageProcessor.from_pretrained("google/efficientnet-b4")
# teacher = EfficientNetForImageClassification.from_pretrained("google/efficientnet-b4")

# processor = AutoImageProcessor.from_pretrained("ahsanjavid/convnext-tiny-finetuned-cifar10")
# teacher = AutoModelForImageClassification.from_pretrained("ahsanjavid/convnext-tiny-finetuned-cifar10")

# processor = AutoImageProcessor.from_pretrained("aaraki/vit-base-patch16-224-in21k-finetuned-cifar10")
# teacher = AutoModelForImageClassification.from_pretrained("aaraki/vit-base-patch16-224-in21k-finetuned-cifar10")
# feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")

/opt/homebrew/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/opt/homebrew/anaconda3/lib/python3.11/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [4]:
print(sum(p.numel() for p in teacher.parameters()))

85806346


In [5]:
student = Student(3, 32, 10, 0.2)
student.load_state_dict(torch.load('model/cifar10_github/epoch_99.bin'))
# model.eval()

<All keys matched successfully>

In [6]:
from torchvision.transforms import ToTensor, Normalize, Resize, Compose
import torchvision.transforms as transforms
import torchvision.datasets as datasets

transforms_cifar = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

dataset = {}

dataset['train'] = datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms_cifar)
dataset['test'] = datasets.CIFAR10(root='./data', train=False, download=True, transform=transforms_cifar)
dataset['raw_train'] = datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms.ToTensor())


# split = dataset['train'].train_test_split(test_size=(5000.0/50000), seed=42)

# dataset['splitted_train'] = split['train']
# dataset['validation'] = split['test']

preprocessed_dataloaders={}
preprocessed_dataloaders['train'] = torch.utils.data.DataLoader(dataset['train'], batch_size=batch_size)
preprocessed_dataloaders['test'] = torch.utils.data.DataLoader(dataset['test'], batch_size=test_batch_size)
preprocessed_dataloaders['raw_train'] = torch.utils.data.DataLoader(dataset['raw_train'], batch_size=batch_size)

# preprocessed_dataloaders['validation'] = torch.utils.data.DataLoader(dataset['validation'], batch_size=test_batch_size)



# dataset = load_dataset("cifar10")
# split = dataset['train'].train_test_split(test_size=(5000.0/50000), seed=42)
# # split2 = dataset['train'].train_test_split(test_size=(5000.0/49000))
# dataset['splitted_train'] = split['train']
# # dataset['extra'] = split2['train']
# dataset['validation'] = split['test']


# def preprocess_images(examples):
#     # get batch of images
#     images = examples['img']
#     examples['img'] = [np.array(image) for image in examples['img']]
#     # convert to list of NumPy arrays of shape (C, H, W)
#     images = [np.array(image, dtype=np.uint8) for image in images]
#     images = [np.moveaxis(image, source=-1, destination=0) for image in images]
#     # preprocess and add pixel_values
#     inputs = feature_extractor(images=images)
#     examples['pixel_values'] = inputs['pixel_values']
#     return examples

# features = Features({
#     'label': ClassLabel(names=['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']),
#     # 'img': Image(decode=True, id=None),
#     # could probably change img to int for faster inference
#     'img': Array3D(dtype="float32", shape=(3,32,32)),
#     'pixel_values': Array3D(dtype="float32", shape=(3, 224, 224)), })

# # preprocessed_train = dataset['train'].map(preprocess_images, batched=True, features=features)
# preprocessed_val = dataset['validation'].map(preprocess_images, batched=True, features=features)
# # preprocessed_test = dataset['test'].map(preprocess_images, batched=True, features=features)
# preprocessed_splitted_train = dataset['splitted_train'].map(preprocess_images, batched=True, features=features)

# # set format to PyTorch
# # preprocessed_train.set_format('torch', columns=['img', 'pixel_values', 'label'])
# preprocessed_val.set_format('torch', columns=['img', 'pixel_values', 'label'])
# # preprocessed_test.set_format('torch', columns=['img', 'pixel_values', 'label'])
# preprocessed_splitted_train.set_format('torch', columns=['img', 'pixel_values', 'label'])

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [7]:
# preprocessed_dataloaders = {}
# preprocessed_dataloaders['splitted_train'] = torch.utils.data.DataLoader(preprocessed_splitted_train, batch_size=batch_size, shuffle=True)
# preprocessed_dataloaders['validation'] = torch.utils.data.DataLoader(preprocessed_val, batch_size=batch_size)
# preprocessed_dataloaders['test'] = torch.utils.data.DataLoader(preprocessed_test, batch_size=test_batch_size)

In [8]:
# import time
# teacher_input = next(iter(preprocessed_dataloaders['splitted_train']))
# start = time.time()
# with torch.no_grad():
#     teacher_output = teacher(teacher_input['pixel_values'])
# total = time.time() - start
# print(total)
# print(teacher_output)

In [9]:
# teacher_saved = torch.load(os.path.join('inference', 'splitted_train.pt'))[0][1]
# print(teacher)
# teacher_input = preprocessed_dataloaders['splitted_train'].dataset
# # print(teacher(teacher_input))
# # print(teacher_input['label'])

In [10]:
# import time
# student_input = next(iter(preprocessed_dataloaders['test']))  
# start = time.time()
# student_output = student(student_input['img'])
# total = time.time()-start
# print(total)
# print(student_output)

In [12]:
loss_calculator = LossCalulcator(temperature, alpha)
optimizer = optim.Adam(
    student.parameters(),
    lr=lr)
# scheduler = optim.lr_scheduler.StepLR(optimizer = optimizer,
                                    # step_size = lr_stepsize)

In [24]:
# # prepare teacher, only need to run once
# teacher_outputs = []
# with torch.no_grad():
#     for images,labels in (preprocessed_dataloaders['raw_train']):
#         # images = [np.array(image, dtype=np.uint8) for image in images]
#         # images = [np.moveaxis(image, source=-1, destination=0) for image in images]
#         teacher_inputs = feature_extractor(images=images, return_tensors='pt', do_rescale=False)
#         teacher_outputs.append(teacher(**teacher_inputs).logits)

# torch.save(teacher_outputs, os.path.join('inference', 'train.pt'))

In [ ]:
train(student = student,
    teacher = teacher,
    dataloader = preprocessed_dataloaders['train'],
    val_dataloader = preprocessed_dataloaders['test'],
    optimizer = optimizer,
    # scheduler = scheduler,
    loss_calculator = loss_calculator,
    epochs = epochs,
    device = device,
    )

In [ ]:
# preprocessed_test = dataset['test'].map(preprocess_images, batched=True, features=features)
# preprocessed_test.set_format('torch', columns=['img', 'pixel_values', 'label'])
# preprocessed_dataloaders['test'] = torch.utils.data.DataLoader(preprocessed_test, batch_size=test_batch_size)

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()
evaluate_model(preprocessed_dataloaders['test'], student, loss_fn, device)